In [ ]:
import soepdoku as soep
from os.path import join
from soepversionizer import Database, Versionizer

path1 = "C:/Users/chh/Work/02_git/Dokumentation/"  # path to SOEP-Core documentation
path2 = "C:/Users/chh/Work/02_git/soep-lee2-compare/" # path to SOEP-LEE2-Compare documentation
version = 'v39'

questionnaire_files = [
    join(path1, 'questionnaires/soep-core-2021-lee2estab/questions.csv'),                           # SOEP-Betriebe, questionnaires
    join(path1, 'questionnaires/soep-core-2023-lee2estab/questions.csv'),
    join(path1, 'questionnaires/soep-core-2020-selfempl/questions.csv'),                            # SOEP-Selbstständinge, questionnaires
    join(path1, 'questionnaires/soep-core-2022-selfempl/questions.csv'),
    join(path1, 'questionnaires/soep-core-2024-selfempl/questions.csv'),
    join(path2, 'questionnaires/soep-lee2-compare-2022-estab/questions.csv'),                       # IAB-Betriebe, questionnaires
    join(path2, 'questionnaires/soep-lee2-compare-2023-estab/questions.csv'),
    join(path2, 'questionnaires/soep-lee2-compare-2024-estab/questions.csv'),
]


doku_repos = [path1, path2]
database = Database(paths = questionnaire_files, doku_repos=doku_repos, version=version)
database.build()
questions, quest_expl = database.merge_quest_log_gen()

Versionizer(
    questionnaire_files=questionnaire_files, 
    doku_repos=[path1, path2], 
    version='v39', 
    build_db_at_start=False, 
    quest_expl=quest_expl, 
    questions=questions
)

In [6]:
import tkinter as tk
from tkinter import ttk
from soepversionizer.const import PADDINGS

class FileMenu(ttk.Combobox):

    def __init__(self, root, variable, col=0, row=0, output=None, **kwargs):
        super().__init__(**kwargs)
        self.root = root

test = FileMenu("root", "test")

In [4]:
Versionizer(
    questionnaire_files=questionnaire_files, 
    doku_repos=[path1, path2], 
    version='v39', 
    build_db_at_start=False, 
    quest_expl=quest_expl, 
    questions=questions
)


In [12]:
questions = soep.read_csv(join(path1, 'questionnaires/soep-core-2021-lee2estab/questions.csv'))

groups = questions.groupby(['study', 'questionnaire', 'question']).size().reset_index()[['study', 'questionnaire', 'question']]
print(type(groups))


for g in groups.iterrows():
    study = g[1]['study']
    questionnaire = g[1]['questionnaire']
    question_number = g[1]['question']

    data = questions.loc[(
    questions['study']==study) &
    (questions['questionnaire']==questionnaire) & 
    (questions['question']==question_number)]

    print(data.loc[data['answer_list']!="", ['item', 'answer_list']])
    print(" ")
    #for i in data.iterrows():
        #print(i[1]['item'])
    

<class 'pandas.core.frame.DataFrame'>
       item answer_list
1  art_betr  10037M4652
 
Empty DataFrame
Columns: [item, answer_list]
Index: []
 
Empty DataFrame
Columns: [item, answer_list]
Index: []
 
Empty DataFrame
Columns: [item, answer_list]
Index: []
 
         item answer_list
80   herausf1  10039M4652
81   herausf2  10039M4652
82   herausf3  10039M4652
83   herausf4  10039M4652
84   herausf5  10039M4652
85   herausf6  10039M4652
86   herausf7  10039M4652
87   herausf8  10039M4652
88   herausf9  10039M4652
89  herausf10  10039M4652
 
           item answer_list
92  personalabt      1M4652
 
Empty DataFrame
Columns: [item, answer_list]
Index: []
 
                 item answer_list
95  personalabt_vorst      1M4652
 
              item answer_list
96  personalstelle      1M4652
 
                       item answer_list
97  personalabt_oberstvorst  10040M4652
 
            item answer_list
98  personalplan      1M4652
 
           item answer_list
99  planungshor  10041M4652
 
    

In [14]:
def my_func(bl):
    return not bl

my_func(False)

True

In [4]:
# DataBase.py


paths = get_paths("Q:/soep-lee2-compare/", col_value={'questionnaire': 'soep-lee2-compare-2022-estab'})
#paths = get_paths("Q:/Dokumentation/", col_value={'questionnaire': 'soep-core-2021-lee2estab'})
paths = get_paths("Q:/soep-lee2-compare/", filename='generations.csv', include_version_dir=['v39'])

for p in paths:
    print(p)

Q:/soep-lee2-compare/datasets\estab2022\v39\generations.csv


In [62]:
for root, dirs, files in os.walk('Q:/Dokumentation/datasets/pl/', topdown=True):
    
    basename = os.path.basename(root)

    if (not is_version_dir(basename)):
        print(root, dirs, files)
    
    

Q:/Dokumentation/datasets/pl/ ['v34', 'v35', 'v36', 'v37', 'v38', 'v39', 'v39.1', 'v40'] ['codebook_alt.csv']


In [8]:
path1 = "C:/Users/chh/Work/02_git/Dokumentation/"  # path to SOEP-Core documentation
path2 = "C:/Users/chh/Work/02_git/soep-lee2-compare/" # path to SOEP-LEE2-Compare documentation
version = 'v39'

file = join(path1, 'questionnaires/soep-core-2021-lee2estab/questions.csv')
#quest = soep.read_csv(join(path1, 'questionnaires/soep-core-2021-lee2estab/questions.csv'))
#answers = soep.read_csv(join(path1, 'questionnaires/soep-core-2021-lee2estab/answers.csv'))
from pathlib import Path
from os.path import join
answers_file = join(Path(file).parents[0], 'answers.csv')
print(answers_file)

answers = soep.read_csv(answers_file)

answers = answers.head(10)


C:\Users\chh\Work\02_git\Dokumentation\questionnaires\soep-core-2021-lee2estab\answers.csv


In [45]:
def add_cols(series):
    return '['+ series['value'] + '] ' + series['label_de']


test = answers[['value', 'label_de']].apply(lambda row: add_cols(row), axis=1)

print(', '.join(test.values))

[1] … ein unabhängiges, eigenständiges Unternehmen oder eine eigenständige Organisation ohne Niederlassung an anderer Stelle?, [2] … die Zentrale oder Hauptverwaltung eines Unternehmens oder einer Organisation mit Niederlassungen (Betrieben / Dienststellen) anderswo?, [3] … eine Niederlassung (Betrieb / Dienststelle) eines größeren Unternehmens oder einer größeren Organisation?, [4] … eine regionale oder fachliche Niederlassung / Mittelinstanz eines verzweigten Unternehmens oder einer verzweigten Organisation?, [-1] Kann / möchte ich nicht beantworten, [1] Keine Herausforderung, [2] Geringe Herausforderung, [3] Mittlere Herausforderung, [4] Große Herausforderung, [5] Sehr große Herausforderung


In [3]:
str1 = "This is \na test\nSo good"
print(str1.split('\n'))

['This is ', 'a test', 'So good']


In [15]:

file = files[0]

def window1():
    layout = [
        [sg.Text("Select a file"), sg.Combo(files, key='-COMBO-', enable_events=True)],
        [sg.Text(f"Selected file:"), sg.Text("", key='-FILE-')],
        [sg.Button("Read dataset")],
        [sg.Text("Result:"), sg.Text("", key='-RESULT-', size=(15, 1))],
        [sg.Button("Go to Window 2"), sg.Button("Exit")]
    ]

    return sg.Window("Window 1", layout)


def select_file(window):
    file = values['-COMBO-']
    window.update(file)
    return file

def read_data(window):
    try:
        data = soep.read_csv(file)
        col = data.columns[3]
        value = data.at[0, col]
        window.update(value)
    except:
        pass


window = window1()

while True:
    event, values = window.read()

    if event in (sg.WINDOW_CLOSED, "Exit"):
        break

    elif event == "-COMBO-":
        file = select_file(window['-FILE-'])
        read_data(window['-RESULT-'])

    elif event == "Read dataset":
        read_data(window['-RESULT-'])
       


window.close()

In [ ]:
def window2():
    layout = [
        [sg.Text("Welcome to window 2")],
        [sg.Button("Go to Window 1"), sg.Button("Exit")]
    ]

    return sg.Window("Window 1", layout)

    elif event == "Calculate Square":
        try:
            number = float(values['-INPUT-'])
            result = number ** 2
            window['-RESULT-'].update(result)

        except ValueError:
            pass


In [ ]:
class Algorithms():

    def __init__(self, questionnaire, paths, doku_repos, questionnaire_path=None):

        self.questionnaire = questionnaire
        self.questionnaire_path = self.get_questionnaire_path(questionnaire_path)
        self.paths = paths # paths to all relevant questionnaires
        self.doku_repos = doku_repos # paths to Documentation top level, ex.: ['C:/Dokumentation', 'C:/Dokumentation_2']

    def get_questionnaire_path(self, path):
        
        if path is None:
            try:
                return self.questionnaire.path
            except:
                return None
        else:
            return path